In [ ]:
import rclpy as rp
import jupyros.ros2 as jr2
import jupyros.ros2.turtle_sim as turtle
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
import os
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sidecar import Sidecar
from time import time, sleep
import math



In [ ]:
# Initialize ROS communications for a given context
if(rp.ok() == False):
    rp.init()
else:
    print("rclpy already initiated")

In [ ]:
superturtle = rp.create_node("superturtle")
moveNode = rp.create_node("moveNode")

In [ ]:
turtlesim = turtle.TurtleSim(background_color="#0000FF")

In [ ]:
display(turtlesim.canvas)

**TIP:** When using JupyterLab, you can right-click on the canvas and select *Create New View from Output*

In [ ]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

In [ ]:
topic_name = '/Pose'
def move_turtles(msg):
    scale = 0.0015
    name = "turtle1"
    
    def angle_slope():
        d_y = (math.sin(msg.theta+1/180)*math.cos(msg.theta+1/180)-math.sin(msg.theta)*math.cos(msg.theta))
        d_x = (math.cos(msg.theta+1/180) - math.cos(msg.theta))
        return math.atan2(d_y,d_x)
        
    poses[name] = {"x": 1/scale * math.cos(msg.theta) + 800,
                   "y": -1/scale * math.sin(msg.theta)*math.cos(msg.theta) + 800,
                   "theta": angle_slope()}
    ##msg.theta - math.atan2((-1/scale * math.sin(msg.theta)*math.cos(msg.theta) + 800),(1/scale * math.cos(msg.theta) + 800))
  
    turtlesim.move_turtles(new_poses=poses)




def cb(msg):
    move_turtles(msg)


turtle_control = jr2.Subscriber(moveNode, Pose, topic_name, cb)
turtle_control.display()

In [ ]:
%%thread_cell2
run = True
i = 90
pub = jr2.Publisher(moveNode, Pose, topic_name)

while run:
    msg = Pose()
    msg.theta = i / 180 * math.pi
    pub.send_msg( msg)
    sleep(0.01)
    i += 1
print("Done")